In [1]:
import torch
import os
import matplotlib.pyplot as plt
os.chdir("..")

from epilearn.models.SpatialTemporal.STGCN import STGCN
from epilearn.models.SpatialTemporal.MepoGNN import MepoGNN
from epilearn.models.SpatialTemporal.EpiGNN import EpiGNN
from epilearn.models.SpatialTemporal.DASTGN import DASTGN
from epilearn.models.SpatialTemporal.ColaGNN import ColaGNN
from epilearn.models.SpatialTemporal.EpiColaGNN import EpiColaGNN
from epilearn.models.SpatialTemporal.CNNRNN_Res import CNNRNN_Res
from epilearn.models.SpatialTemporal.ATMGNN import MPNN_LSTM, ATMGNN

from epilearn.models.Temporal.Dlinear import DlinearModel
from epilearn.models.Temporal.LSTM import LSTMModel
from epilearn.models.Temporal.GRU import GRUModel

from epilearn.data import UniversalDataset
from epilearn.utils import utils, transforms
from epilearn.tasks.forecast import Forecast

### Configs

In [2]:
# initial settings
device = torch.device('cpu')
torch.manual_seed(7)

lookback = 12 # inputs size
horizon = 3 # predicts size

# permutation is True when using STGCN
permute = False

epochs = 10 # training epochs
batch_size = 50 # training batch size

### Initialize dataset

In [3]:
# load toy dataset
dataset = UniversalDataset()
dataset.load_toy_dataset()

### Initialize model and task
* prototype supports all models imported at the first cell

In [4]:
task = Forecast(prototype=EpiGNN, dataset=None, lookback=lookback, horizon=horizon, device='cpu')

### Add transformations to dataset

In [5]:
transformation = transforms.Compose({"features":[transforms.normalize_feat()], 
                                 'graph': [transforms.normalize_adj()], 
                                 'dynamic_graph': [transforms.normalize_adj()], 
                                 'states': []
                                 })
dataset.transforms = transformation

In [6]:
dir(dataset)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'download',
 'dynamic_graph',
 'edge_attr',
 'edge_index',
 'edge_weight',
 'generate_dataset',
 'get_slice',
 'get_transformed',
 'graph',
 'load_toy_dataset',
 'output_dim',
 'save',
 'states',
 'transforms',
 'x',
 'y']

### Train Model
* for epicolagnn, loss='epi_cola' else loss='mse
* for STGCN, permute_dataset=True

In [7]:
config = None
result = task.train_model(dataset=dataset, config=config, loss='mse', epochs=5, permute_dataset=permute) # instead of config, we can also dircetly input some parameters

spatial-temporal model loaded!


 40%|████      | 2/5 [00:00<00:00,  4.55it/s]

######### epoch:0
Training loss: 159.5062492124496
Validation loss: 0.6187673807144165


100%|██████████| 5/5 [00:01<00:00,  4.71it/s]



Final Training loss: 12.74922546263664
Final Validation loss: 0.6182436943054199
Test MAE: 201.59722900390625
Test RMSE: 539.6631469726562


### Evaluate model

In [8]:
evaluation = task.evaluate_model()

Test MAE: 201.59722900390625
Test RMSE: 539.6631469726562


### Try more datasets

In [9]:
# load other datasets
datasets = [dataset]
raw_data = torch.load("datasets/benchmark.pt")
for name in ['Brazil', 'Austria', 'China']:
    data = raw_data[name]
    dataset = UniversalDataset()
    dataset.x = data['features']
    dataset.y = data['features'][:,:,0]
    dataset.graph = data['graph']
    dataset.states = data['features']
    dataset.dynamic_graph = None

    dataset.transforms = transformation
    datasets.append(dataset)


In [10]:
for i, dataset in enumerate(datasets):
    print(f"dataset {i}")
    model = task.train_model(dataset=dataset, config=config, loss='mse', epochs=50, batch_size=50, permute_dataset=permute) # instead of config, we can also dircetly input some parameters

dataset 0
spatial-temporal model loaded!


  4%|▍         | 2/50 [00:00<00:08,  5.34it/s]

######### epoch:0
Training loss: 157.28934587751115
Validation loss: 6.223514080047607


100%|██████████| 50/50 [00:09<00:00,  5.55it/s]




Final Training loss: 0.3075927325657436
Final Validation loss: 0.4116224944591522
Test MAE: 190.63783264160156
Test RMSE: 545.3310546875
dataset 1
spatial-temporal model loaded!


...



Final Training loss: 1.8250496685504913
Final Validation loss: 0.15613172948360443
Test MAE: 26.055709838867188
Test RMSE: 29.661773681640625
dataset 3
spatial-temporal model loaded!


 10%|█         | 5/50 [00:00<00:01, 42.35it/s]

######### epoch:0
Training loss: 11.228009223937988
Validation loss: 9.210454940795898


100%|██████████| 50/50 [00:01<00:00, 42.16it/s]



Final Training loss: 1.4774788618087769
Final Validation loss: 0.5533209443092346
Test MAE: 132.78994750976562
Test RMSE: 959.5199584960938


### Try temporal models

In [11]:
task = Forecast(prototype=LSTMModel, dataset=None, lookback=lookback, horizon=horizon, device='cpu')
num_nodes = 47
mae_list=[]
rmse_list=[]
for region in range(num_nodes):
    print("region", region)
    result = task.train_model(dataset=datasets[-1], config=config, loss='mse', epochs=50, batch_size=50, region_idx=1, permute_dataset=False)
    mae_list.append(result['mae'])
    rmse_list.append(result['rmse'])

mae = torch.FloatTensor(mae_list)
rmse = torch.FloatTensor(rmse_list)
print(f"mae:{mae.mean()} {mae.std()}")
print(f"rmse:{rmse.mean()} {rmse.std()}")

region 0
temporal model loaded!


 18%|█▊        | 9/50 [00:00<00:00, 87.66it/s]

######### epoch:0
Training loss: 0.0067208558320999146
Validation loss: 0.003996503539383411


100%|██████████| 50/50 [00:00<00:00, 102.61it/s]




Final Training loss: 0.00011676810390781611
Final Validation loss: 0.000115883449325338
Test MAE: 3.174466133117676
Test RMSE: 3.8885982036590576
region 1
temporal model loaded!


...



Final Training loss: 0.00011967199679929763
Final Validation loss: 0.00010306303738616407
Test MAE: 3.0903968811035156
Test RMSE: 3.9795448780059814
region 46
temporal model loaded!


 18%|█▊        | 9/50 [00:00<00:00, 85.38it/s]

######### epoch:0
Training loss: 0.006112487055361271
Validation loss: 0.003542650956660509


100%|██████████| 50/50 [00:00<00:00, 91.14it/s]



Final Training loss: 0.0001338159927399829
Final Validation loss: 0.00011659053416224197
Test MAE: 3.0903539657592773
Test RMSE: 3.9273111820220947
mae:3.132838726043701 0.05110329017043114
rmse:4.013798236846924 0.1063409224152565
